# Visualizing the GAN results

Let's visualize some random results from the GAN. To do so we just need to load a generator model and feed it with random Gaussian noise of the approriate size

## Visualizing the results in a notebook

First let's load the generator. You need to watch out for three variables here:
- `latent_dim`
- `model_path`
- `epoch`

The first variable, `latent_dim` defines the dimension of your ["latent vector"](https://medium.com/@jain.yasha/gan-latent-space-1b32cd34cfda). If you changed the same variable in the training notebook, you will have to change it here as well.

The second, `model_path` defines the directory where you saved your models.

The third, `epoch` defines the epoch for which you want to load a model. You can examine the directory and the example images for each epoch, to choose which epoch you want to visualize.

In [ ]:
import os 
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
model_dir = './models/dcgan_celeba'
generator_path = os.path.join(model_dir, "e060_generator_celeba.keras")
generator = tf.keras.models.load_model(generator_path)

In [ ]:
# utils
# note the image resizing (see https://www.tensorflow.org/api_docs/python/tf/image/resize)

def denorm(x):
    """Denormalize the outputs from [-1, 1] to [0,255] (generator with 'tanh' activation)"""
    return (x + 1) * 127.5

def generate(generator, n=8, resizing=(254,254)):
    latent_dim = generator.input_shape[1] # the input shape is (batch, latent_dim)
    random_latent_vectors = tf.random.normal(shape=(n, latent_dim))
    generated_images = generator(random_latent_vectors, training = False)
    generated_images = tf.image.resize(generated_images, resizing) # resizing our images
    return tf.cast(denorm(generated_images), tf.uint8)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_gan_images(generated_images, max_cols=4, fig_w=4, fig_h=4):
    """
    Adaptable plotting function handling any number of images,
    (rewritten by ChatGPT, with some requests for modifications, on the basis of 
    a function inspired by: https://stackoverflow.com/a/54681765)
    """
    n_images = generated_images.shape[0]
    
    nrows = int(np.ceil(n_images / max_cols)) # Calculate the number of rows and columns
    ncols = min(n_images, max_cols)            # This handles cases where there are fewer images than max_cols
    
    fig, axs = plt.subplots(nrows, ncols, figsize=(fig_w*ncols, fig_h*nrows))
    
    # If there's only one row or one column, make sure axs is still a 2D array for consistency
    if nrows == 1:
        axs = np.expand_dims(axs, axis=0)
    if ncols == 1:
        axs = np.expand_dims(axs, axis=1)

    # Turn off the axes for all subplots initially
    for ax in axs.ravel():
        ax.axis('off')

    # Then only display the images on the subplots that have them
    for img, ax in zip(generated_images, axs.ravel()):
        ax.imshow(img.numpy())

    plt.show()

plot_gan_images(generate(generator))

## Random Walk

We can use a loop to gradually add some random noise to our latent vector, effectively 'moving' (blindly, chaotically) in the latent space.

In [ ]:
def generate_random_walk(generator, n=8, resizing=(256,256)):
    latent_dim = generator.input_shape[1] # the input shape is (batch, latent_dim)
    a = tf.random.normal(shape=(latent_dim,))
    random_latent_vectors = [a]
    for t in range(n):
        noise =  tf.random.normal(shape=a.shape, mean=0, stddev=0.2) # Gaussian/Bell Curve: try tweaking the stddev
        # noise =  tf.random.uniform(shape=a.shape, minval=-.3, maxval=.3) # White Noise: try tweaking the min/max values
        random_latent_vectors.append(random_latent_vectors[-1] + noise)
    random_latent_vectors = tf.stack(random_latent_vectors)
    generated_images = generator(random_latent_vectors, training = False)
    generated_images =  tf.image.resize(generated_images, resizing)
    return tf.cast(denorm(generated_images), tf.uint8)

plot_gan_images(generate_random_walk(generator, 100), max_cols=10, fig_w=2, fig_h=2)

## Interpolating in latent space

We can interpolate between one point in the latent space (the variable `a`) and another point (the variable `b`) to produce a smooth transition between images generated by the GAN along the latent space. It is recommended to use a geod "spherical linear interpolation", which effectively describes a ["geodesic"](https://en.wikipedia.org/wiki/Geodesic) ([mini-vid](https://www.youtube.com/watch?v=KsdIuVByfMc)). We use spherical interpolation because the multivariate Gaussian used as an input to the GAN generator can be approximated by a hypersphere (a sphere in high dimensions).

See [this discussion](https://github.com/soumith/dcgan.torch/issues/14) and [this post](https://machinelearningmastery.com/how-to-interpolate-and-perform-vector-arithmetic-with-faces-using-a-generative-adversarial-network/) for technical details and to see where the interpolation code comes from.

In [ ]:
from numpy.linalg import norm

def lerp(t, a, b):
    return a + t*(b - a)

def slerp(val, low, high):
    omega = np.arccos(np.clip(np.dot(low/norm(low), high/norm(high)), -1.0, 1.0))
    so = np.sin(omega)
    if so == 0:
        # L'Hopital's rule/LERP
        return (1.0-val) * low + val * high
    return np.sin((1.0-val)*omega) / so * low + np.sin(val*omega) / so * high

In [ ]:
def generate_interpolated(generator, n=8, resizing=(256,256)):
    latent_dim = generator.input_shape[1] # the input shape is (batch, latent_dim)
    a = tf.random.normal(shape=(latent_dim,))
    b = tf.random.normal(shape=(latent_dim,))
    random_latent_vectors = []
    for t in np.linspace(0, 1, n):
        random_latent_vectors.append(slerp(t, a, b))
    random_latent_vectors = tf.stack(random_latent_vectors)
    generated_images = generator(random_latent_vectors, training = False)
    generated_images =  tf.image.resize(generated_images, resizing)
    return tf.cast(denorm(generated_images), tf.uint8)

plot_gan_images(generate_interpolated(generator, 100), max_cols=10, fig_w=2, fig_h=2)

In case, a tutorial with a few other approaches: [Generate Artificial Faces with CelebA Progressive GAN Model](https://www.tensorflow.org/hub/tutorials/tf_hub_generative_image_module).